# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 20.09.2022

Мягкий дедлайн: 23:59MSK 04.10.2022

Жесткий дедлайн: 23:59MSK 10.10.2022

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import pandas as pd
import warnings
import torch.utils.data as data_utils
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np 
import torch
import torch.nn.functional as F
import torchvision
from IPython.display import clear_output
from torch import nn, optim
from torchvision import transforms
from tqdm.auto import tqdm, trange
import random
from sklearn.preprocessing import StandardScaler

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 15.7 MB/s 
     |████████████████████████████████| 182 kB 41.6 MB/s 
     |████████████████████████████████| 162 kB 16.6 MB/s 
     |████████████████████████████████| 63 kB 470 kB/s 
     |████████████████████████████████| 162 kB 74.4 MB/s 
     |████████████████████████████████| 158 kB 62.6 MB/s 
     |████████████████████████████████| 157 kB 75.7 MB/s 
     |████████████████████████████████| 157 kB 71.6 MB/s 
     |████████████████████████████████| 157 kB 75.7 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 157 kB 78.9 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 156 kB 78.0 MB/s 


In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: aiyufanov. Use `wandb login --relogin` to force relogin


True

In [ ]:
# %conda install wget

In [5]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2022-10-10 19:10:23--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘data.txt.zip’

data.txt.zip        100%[===================>] 201.24M  16.4MB/s    in 14s     

2022-10-10 19:10:38 (14.7 MB/s) - ‘data.txt.zip’ saved [211011981/211011981]



In [6]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [18]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]


In [ ]:
y_train.shape

(463715,)

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [19]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
from sklearn.linear_model import Ridge
clf_ridge = Ridge(alpha = 0.8)
clf_ridge.fit(X_train, y_train)
y_pred_ridge = clf_ridge.predict(X_train)
y_pred1_ridge = clf_ridge.predict(X_test)
print(np.sqrt(mean_squared_error(y_train, y_pred_ridge)))
print(np.sqrt(mean_squared_error(y_test, y_pred1_ridge)))


9.552822967904051
9.51016071059644


## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [20]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [21]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
set_random_seed(107)
y_train = y_train.reshape(-1,1)
scaler = StandardScaler()
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_train = y_train.reshape(463715,)


In [22]:
from torch.nn.modules import dropout
set_random_seed(107)



class RandomDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      if scale_data:
          X = StandardScaler().fit_transform(X)
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

class ModelDZ1(nn.Module):
  def __init__(self):
    super(ModelDZ1, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(90,70),
        #nn.Dropout(),
        nn.Sigmoid(),
        nn.Linear(70,60),
        nn.Sigmoid(),
        nn.Linear(60,32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Linear(16,1)
    )
  def forward(self, x):
    return self.net(x)

train_set =  RandomDataset(X_train, y_train)  # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True, num_workers=1) # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ

test_set = RandomDataset(X_test, y_test) # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
test_loader =  torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1) # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ

model = ModelDZ1() # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
criterion = nn.MSELoss() # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ



In [23]:
wandb.init(project="test4")

mean val loss,▁
mean val loss,1998.44084


In [24]:
wandb.watch(model);


In [29]:
set_random_seed(107)
from torch.nn.modules.loss import MSELoss

def train(model, optimizer, criterion, train_loader,  test_loader, scaler):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    for e in range(13):
      train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
      for i, data in tqdm(enumerate(train_loader, 0)):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        #print(inputs)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
      if e  % 2 == 0:

          val_loss = []                                  
          pred_arr = []
          y_arr = []
          with torch.no_grad(): 
            
            for i, data in tqdm(enumerate(test_loader, 0)):
              inputs1, targets1 = data
              inputs1, targets1 = inputs1.float(), targets1.float()
              targets1 = targets1.reshape((targets1.shape[0], 1))
              y_pred = model(inputs1)
              y_arr.append(targets1[0][0].item())
              pred_arr.append(y_pred[0][0].item())
            pred_arr = np.array(pred_arr)
             
            pred_arr = pred_arr.reshape(-1,1)
            pred_arr = scaler.inverse_transform(pred_arr)
            #print(pred_arr)

            pred_arr = pred_arr.reshape(51630,)
            #print(torch.tensor(pred_arr[0]))
            y_arr = torch.tensor(y_arr, dtype = torch.float64)

            wandb.log({"mean val loss": np.mean(np.sqrt(mean_squared_error(y_arr,pred_arr)))})
           # print(y_arr)
          #  print(pred_arr)

            #print(criterion(torch.tensor(y_pred), torch.tensor(y_arr)))
            #val_loss.append(criterion(y_pred, y_arr))
              
          # скидываем метрики на wandb и автоматом смотрим на графики
            
          
            #print(f"Epoch: {e}, loss: {np.mean(val_loss)}")

      # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
     

      
def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------

    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    with torch.no_grad():

      pred_arr = []
      predicts = torch.ones(len(test_loader.dataset))
      loss_array = []
      for i, data in tqdm(enumerate(test_loader, 0)):
          inputs, targets = data
          inputs, targets = inputs.float(), targets.float()
          targets = targets.reshape((targets.shape[0], 1))
          #print(inputs)
        

          outputs = model(inputs)
          pred_arr.append(outputs)

      pred_arr = np.array(pred_arr)
             
      pred_arr = pred_arr.reshape(-1,1)
      pred_arr = scaler.inverse_transform(pred_arr)
      pred_arr = torch.tensor(pred_arr)
          #loss = criterion(outputs, targets)
          #loss_array.append(loss)
      # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
      
      #print(predicted_array) 

    return (pred_arr)

In [30]:
pred = test(model,criterion, test_loader)
pred1 = torch.tensor(pred)

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

In [31]:
assert pred1.shape[0] == y_test.shape[0]

In [32]:
print(np.sqrt(mean_squared_error(pred, y_test)))

8.849713910943537


## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

In [ ]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
'''
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) (В качестве оптимайзера во всех случаях использовался Adam)
Во всех тестах и на валидации одна метрика - RMSE
1 эксперимент: https://wandb.ai/aiyufanov/test1/runs/2wsxgsst?workspace=user-
Для нормировки таргета использовался StandartScaler, для тренировки был выбран батч, равный 10, количество эпох - 10.
В Wandb предствлен график RMSELoss'а. Валидация проводилась каждые 2 эпохи. Также был использован Dropout
random_seed = 100
Loss на тестовой выборке составил 8.905
Функции активации: первая ReLU, Вторая - Сигмоид
Слои: 1 слой - (90, 150), 2 - (150, 32), 3 - (32, 1)
2 Эксперимент: https://wandb.ai/aiyufanov/test2/runs/3vx737ti?workspace=user-aiyufanov 
Убрал Dropout
Слои: 1 слой - (90, 70), 2 - (70, 32), 3 - (32, 1)
Все функции активации - Сигмоид
random_seed = 105
Loss - 8.882

3 Эксперимент: https://wandb.ai/aiyufanov/test8/runs/3e0yk1o7?workspace=user-aiyufanov
все одинаково с 2 экспериментом, кроме количество эпох
т.к. Loss не начал возрастать(не началось переобучение), количество эпох было увеличино до 15 
Loss - 8.872

4 Эксперимент: https://wandb.ai/aiyufanov/test4/runs/1oz0lgrd?workspace=user-aiyufanov
Поменял Слои: 1 - (90,70), 2 - (70,60), 3 - (60,32), 4 - (32,16), 5 - (16,1)
функции активации: sigmoid, sigmoid, relu, relu
random_seed = 107
LOSS = 8.8497
Эта версия загрудена как решение к дз

'''